In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./datasets/intermediate/shuffled_136k.csv')
df

,URL,URL_Length,Url_Shortening,Ssl_Info,Global_Ranking,Has_Suspicious_Params,Num_Languages,Uses_HTTPS,Has_User_Info,User_Info_Length,...,Num_Dash_Query,Num_AtSymbol_Query,Num_TildeSymbol_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Num_Hash_Query,Hex_Encoded_Characters_Query,Has_Fragment,label
0,https://qrco.de/bfK5aD,22,True,NaN,17503,False,1,1,False,0,...,0,0,0,0,0,0,0,0,False,1
1,idiletisim.com.tr,17,False,NaN,0,False,1,0,False,0,...,0,0,0,0,0,0,0,0,False,0
2,https://t.co/i5Y81ugWlM,23,True,NaN,189,False,1,1,False,0,...,0,0,0,0,0,0,0,0,False,1
3,alamobb.net,11,False,NaN,351109,False,1,0,False,0,...,0,0,0,0,0,0,0,0,False,0
4,https://e-zpassny.com-wwad.xin/,31,False,NaN,0,False,1,1,False,0,...,0,0,0,0,0,0,0,0,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135995,navi-cars.ru,12,False,NaN,4137625,False,1,0,False,0,...,0,0,0,0,0,0,0,0,False,0
135996,https://supportservice23145.weebly.com/,39,False,NaN,298,False,1,1,False,0,...,0,0,0,0,0,0,0,0,False,1
135997,raegunramblings.com,19,False,NaN,390229,False,1,0,False,0,...,0,0,0,0,0,0,0,0,False,0
135998,huddlestonelinens.com,21,False,NaN,-1,False,1,0,False,0,...,0,0,0,0,0,0,0,0,False,0


In [3]:
oldColumns = df.columns
df.sample(frac=1).reset_index(drop=True)

if 'Ssl_Info' in df.columns: 
    df.drop(['Ssl_Info'], axis=1, inplace=True)
    print('Ssl_Info dropped')
if 'Unnamed: 0' in df.columns:
    df.drop(['Unnamed: 0'], axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

nonNumeric = df.select_dtypes(include=['object']).columns
boolCols = df.select_dtypes(include=['bool']).columns
df[boolCols] = df[boolCols].astype(int)

print(nonNumeric)
print(boolCols)

for col in nonNumeric:
    if col != 'URL':
        df[col] = le.fit_transform(df[col])
        print(f'{col} encoded')



Ssl_Info dropped
Index(['URL', 'TLD'], dtype='object')
Index(['Url_Shortening', 'Has_Suspicious_Params', 'Has_User_Info', 'Has_Ip',
       'Has_Port', 'Has_Fragment'],
      dtype='object')
TLD encoded


In [4]:
import joblib
joblib.dump(le, 'saved_models/label_encoder.joblib')

['saved_models/label_encoder.joblib']

In [5]:
df_nourl = df.drop(columns=["URL"])

correlation_with_label = df_nourl.corr(numeric_only=True)["label"].sort_values(ascending=False)

high_corr = correlation_with_label[abs(correlation_with_label) > 0.7]

feature_variance = df_nourl.drop(columns=["label"]).var()
unique_counts = df_nourl.drop(columns=["label"]).nunique()

high_corr, feature_variance.sort_values(), unique_counts.sort_values()

(label            1.000000
 Uses_HTTPS       0.952170
 Num_Dots_Path   -0.818539
 Name: label, dtype: float64,
 Num_TildeSymbol_Subdomain           0.000000e+00
 Num_AtSymbol_Subdomain              0.000000e+00
 Num_Percent_Domain                  0.000000e+00
 Num_TildeSymbol_Domain              0.000000e+00
 Hex_Encoded_Characters_Subdomain    0.000000e+00
                                         ...     
 Length_Query                        1.225015e+03
 URL_Length                          1.205705e+04
 TLD                                 1.156921e+05
 Global_Ranking                      3.032078e+11
 Domain_Age                                   NaN
 Length: 65, dtype: float64,
 Domain_Age                       0
 Num_TildeSymbol_Subdomain        1
 Num_TildeSymbol_Domain           1
 Num_AtSymbol_Subdomain           1
 Num_Ampersand_Subdomain          1
                              ...  
 Length_Path                    207
 Length_Query                   400
 URL_Length           

In [6]:
# Find columns with 0 variance
zero_variance = feature_variance[feature_variance == 0]
zero_variance



Num_AtSymbol_Subdomain              0.0
Num_TildeSymbol_Subdomain           0.0
Num_Percent_Subdomain               0.0
Num_Ampersand_Subdomain             0.0
Num_Hash_Subdomain                  0.0
Hex_Encoded_Characters_Subdomain    0.0
Num_AtSymbol_Domain                 0.0
Num_TildeSymbol_Domain              0.0
Num_Percent_Domain                  0.0
Num_Ampersand_Domain                0.0
Num_Hash_Domain                     0.0
Hex_Encoded_Characters_Domain       0.0
Num_Hash_Path                       0.0
Num_Hash_Query                      0.0
dtype: float64

In [7]:
df_nourl = df_nourl.drop(columns=zero_variance.index)


In [8]:
df_nourl

,URL_Length,Url_Shortening,Global_Ranking,Has_Suspicious_Params,Num_Languages,Uses_HTTPS,Has_User_Info,User_Info_Length,Num_Subdomains,Num_Digits_Subdomain,...,Num_Dots_Query,Num_Dash_Query,Num_AtSymbol_Query,Num_TildeSymbol_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Hex_Encoded_Characters_Query,Has_Fragment,label
0,22,1,17503,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,17,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23,1,189,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,11,0,351109,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,31,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135995,12,0,4137625,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135996,39,0,298,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
135997,19,0,390229,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135998,21,0,-1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
high_corr = correlation_with_label[abs(correlation_with_label) > 0.7]

high_corr.sort_values(ascending=False)

label            1.000000
Uses_HTTPS       0.952170
Num_Dots_Path   -0.818539
Name: label, dtype: float64

In [10]:
df_nourl.drop(['Uses_HTTPS', 'Num_Dots_Path'], axis=1, inplace=True)

df_nourl

unique_counts = df_nourl.nunique()



In [11]:
unique_counts.sort_values(ascending=False)


Global_Ranking                  56419
TLD                              1113
URL_Length                        517
Length_Query                      400
Length_Path                       207
Num_Digits_Query                  165
Num_Digits_Path                    96
Length_Domain                      50
Length_Subdomain                   49
Hex_Encoded_Characters_Query       29
Num_Percent_Query                  29
User_Info_Length                   28
Num_Dash_Query                     27
Num_Underscore_Query               27
Num_Hyphens_Query                  27
Num_Digits_Subdomain               24
Num_Digits_Domain                  23
Num_Query_Params                   23
Num_Ampersand_Query                22
Num_Hyphens_Path                   19
Num_Dots_Query                     17
Num_Underscore_Path                16
Hex_Encoded_Characters_Path        16
Num_Percent_Path                   16
Num_Dash_Path                      16
Path_Level                         15
Num_Hyphens_

In [12]:
nan_counts = df_nourl.isna().sum()
nan_counts.sort_values(ascending=False)

df_nourl.drop(['Domain_Age'], axis=1, inplace=True)


In [13]:
df_nourl


,URL_Length,Url_Shortening,Global_Ranking,Has_Suspicious_Params,Num_Languages,Has_User_Info,User_Info_Length,Num_Subdomains,Num_Digits_Subdomain,Num_Hyphens_Subdomain,...,Num_Dots_Query,Num_Dash_Query,Num_AtSymbol_Query,Num_TildeSymbol_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Hex_Encoded_Characters_Query,Has_Fragment,label
0,22,1,17503,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,17,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23,1,189,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,11,0,351109,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,31,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135995,12,0,4137625,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135996,39,0,298,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
135997,19,0,390229,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135998,21,0,-1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
low_variance = feature_variance[feature_variance < 0.01]
low_variance


Has_Suspicious_Params               0.004247
Has_User_Info                       0.000272
Num_Dots_Subdomain                  0.005245
Num_Dash_Subdomain                  0.000037
Num_AtSymbol_Subdomain              0.000000
Num_TildeSymbol_Subdomain           0.000000
Num_Underscore_Subdomain            0.000037
Num_Percent_Subdomain               0.000000
Num_Ampersand_Subdomain             0.000000
Num_Hash_Subdomain                  0.000000
Hex_Encoded_Characters_Subdomain    0.000000
Has_Ip                              0.004960
Num_Dash_Domain                     0.000029
Num_AtSymbol_Domain                 0.000000
Num_TildeSymbol_Domain              0.000000
Num_Underscore_Domain               0.000029
Num_Percent_Domain                  0.000000
Num_Ampersand_Domain                0.000000
Num_Hash_Domain                     0.000000
Hex_Encoded_Characters_Domain       0.000000
Has_Port                            0.000162
Num_AtSymbol_Path                   0.000544
Num_TildeS

In [15]:
for col in low_variance.index:
    if col in df_nourl.columns:
        df_nourl.drop(col, axis=1, inplace=True)

df_nourl



,URL_Length,Url_Shortening,Global_Ranking,Num_Languages,User_Info_Length,Num_Subdomains,Num_Digits_Subdomain,Num_Hyphens_Subdomain,Length_Subdomain,Num_Digits_Domain,...,Num_Hyphens_Query,Length_Query,Num_Query_Params,Num_Dots_Query,Num_Dash_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Hex_Encoded_Characters_Query,label
0,22,1,17503,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,17,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23,1,189,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,11,0,351109,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,31,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135995,12,0,4137625,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135996,39,0,298,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
135997,19,0,390229,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135998,21,0,-1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# df_nourl.to_csv('./newDatasets/clean_44k.csv', index=False)
low_var = df_nourl.loc[:, df_nourl.nunique() <= 2].columns
print("Low variance features:", low_var.tolist())
correlations = df_nourl.corr(numeric_only=True)["label"].sort_values(ascending=False)
print('Correlations:', correlations)

df_nourl.drop(['Global_Ranking','Path_Level'], axis=1, inplace=True)

Low variance features: ['Url_Shortening', 'Num_Dots_Domain', 'label']
Correlations: label                           1.000000
Path_Level                      0.636664
Url_Shortening                  0.259363
Num_Digits_Path                 0.211755
Num_Query_Params                0.206712
URL_Length                      0.188006
Num_Ampersand_Query             0.169906
Num_Dash_Path                   0.153214
Num_Underscore_Path             0.153214
Length_Query                    0.131891
Num_Subdomains                  0.129625
Num_Hyphens_Domain              0.114458
Num_Hyphens_Path                0.104697
Num_Digits_Query                0.100356
TLD                             0.095166
Num_Dots_Query                  0.088994
Length_Subdomain                0.078812
Num_Dots_Domain                 0.070783
Num_Digits_Subdomain            0.060433
Num_Underscore_Query            0.057636
Num_Dash_Query                  0.057636
Length_Path                     0.055435
Num_Hyphens_Qu

In [17]:
df_nourl.to_csv('./datasets/cleaned/clean_136k.csv', index=False)
df_nourl

,URL_Length,Url_Shortening,Num_Languages,User_Info_Length,Num_Subdomains,Num_Digits_Subdomain,Num_Hyphens_Subdomain,Length_Subdomain,Num_Digits_Domain,Num_Hyphens_Domain,...,Num_Hyphens_Query,Length_Query,Num_Query_Params,Num_Dots_Query,Num_Dash_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Hex_Encoded_Characters_Query,label
0,22,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,17,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,11,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,31,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135995,12,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
135996,39,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
135997,19,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135998,21,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
removed_columns = set(oldColumns) - set(df_nourl.columns)
print("Columns removed from oldColumns to new columns:", removed_columns, '\n', 'Total columns removed:', len(removed_columns))

Columns removed from oldColumns to new columns: {'Num_AtSymbol_Domain', 'Has_User_Info', 'Has_Suspicious_Params', 'Num_TildeSymbol_Domain', 'Num_Dash_Subdomain', 'Num_Hash_Subdomain', 'Num_AtSymbol_Query', 'Num_AtSymbol_Path', 'Num_Hash_Domain', 'Hex_Encoded_Characters_Subdomain', 'Has_Ip', 'Uses_HTTPS', 'Num_Underscore_Subdomain', 'Num_TildeSymbol_Query', 'Num_AtSymbol_Subdomain', 'Num_Hash_Query', 'Ssl_Info', 'Num_Dots_Path', 'Has_Fragment', 'Num_Percent_Subdomain', 'Global_Ranking', 'Num_Percent_Domain', 'Num_TildeSymbol_Subdomain', 'Num_Hash_Path', 'Has_Port', 'Num_Dash_Domain', 'Num_TildeSymbol_Path', 'Num_Ampersand_Subdomain', 'Domain_Age', 'Path_Level', 'Num_Dots_Subdomain', 'Num_Ampersand_Domain', 'Num_Ampersand_Path', 'URL', 'Hex_Encoded_Characters_Domain', 'Num_Underscore_Domain'} 
 Total columns removed: 36


In [19]:
removed_columns = ['Num_Dots_Path','Ssl_Info', 'Num_Dash_Subdomain', 'Path_Level', 'Has_Suspicious_Params', 'Num_Hash_Domain', 'Num_AtSymbol_Subdomain', 'Num_TildeSymbol_Domain', 'Num_Underscore_Subdomain', 'Num_AtSymbol_Path', 'Num_Hash_Path', 'Num_Ampersand_Subdomain', 'Num_DoubleSlash_Path', 'Has_Fragment', 'Num_TildeSymbol_Query', 'URL', 'Num_Hash_Query', 'Hex_Encoded_Characters_Subdomain', 'Has_Port', 'Num_Dots_Subdomain', 'Uses_HTTPS', 'Has_User_Info', 'Global_Ranking', 'Num_Dash_Domain', 'Num_AtSymbol_Domain', 'Domain_Age', 'Num_Ampersand_Domain', 'Num_TildeSymbol_Subdomain', 'Num_Percent_Subdomain', 'Num_Percent_Domain', 'Hex_Encoded_Characters_Domain', 'Has_Ip', 'Num_Underscore_Domain', 'Num_Hash_Subdomain', 'Num_AtSymbol_Query', 'Num_Ampersand_Path', 'Num_TildeSymbol_Path']

val_df = pd.read_csv('./datasets/validation_ds.csv')
val_df.drop(columns=removed_columns, inplace=True)

val_df.sample(frac=1).reset_index(drop=True)

if 'Ssl_Info' in val_df.columns: 
    val_df.drop(['Ssl_Info'], axis=1, inplace=True)
    print('Ssl_Info dropped')
if 'Unnamed: 0' in val_df.columns:
    val_df.drop(['Unnamed: 0'], axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

nonNumeric = val_df.select_dtypes(include=['object']).columns
boolCols = val_df.select_dtypes(include=['bool']).columns
val_df[boolCols] = val_df[boolCols].astype(int)

print(nonNumeric)
print(boolCols)

for col in nonNumeric:
    if col != 'URL':
        val_df[col] = le.fit_transform(val_df[col])
        print(f'{col} encoded')



FileNotFoundError: [Errno 2] No such file or directory: './datasets/validation_ds.csv'

In [20]:
val_df.to_csv('./datasets/cleaned/clean_val.csv', index=False)
val_df

,URL_Length,Url_Shortening,Num_Languages,User_Info_Length,Num_Subdomains,Num_Digits_Subdomain,Num_Hyphens_Subdomain,Length_Subdomain,Num_Digits_Domain,Num_Hyphens_Domain,...,Num_Hyphens_Query,Length_Query,Num_Query_Params,Num_Dots_Query,Num_Dash_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Hex_Encoded_Characters_Query,label
0,175,0,1,0,0,0,0,0,0,0,...,0,43,3,0,0,0,0,2,0,1
1,67,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,23,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,40,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,175,0,1,0,0,0,0,0,0,0,...,0,43,3,0,0,0,0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,10,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
396,13,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
397,7,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
398,12,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
